In [ ]:
%load_ext autoreload
%autoreload 2

In [20]:
import numpy as np
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F

from yolov2tiny import Yolov2
import eecs598
from cnn_scratch import DeepConvNet, Conv, ConvB, Conv_ReLU, Conv_ReLU_Pool, Conv_BatchNorm_ReLU, Conv_BatchNorm_ReLU_Pool
from cnn_torch import FastConv, FastConvWB

In [ ]:
scratch = DeepConvNet(input_dims=(3, 416, 416),
                        num_filters=[16, 32, 64, 128, 256, 512, 1024, 1024],
                        max_pools=[0, 1, 2, 3, 4],
                        weight_scale='kaiming',
                        batchnorm=True,
                        dtype=torch.float32, device='cpu')

class WeightLoader(object):
    def __init__(self):
        super(WeightLoader, self).__init__()
        self.start = 0
        self.buf = None
        self.b = 'b'
        self.g = 'g'
        self.rm = 'rm'
        self.rv = 'rv'
        
    def load_conv_bn(self, conv_model, bn_model):

        num_w = conv_model.weight.numel()
        num_b = bn_model.bias.numel()

        bn_model.bias.data.copy_(
            torch.reshape(torch.from_numpy(self.buf[self.start:self.start + num_b]), bn_model.bias.size()))

        if bn_model.bias.data.shape == scratch.params['beta0'].shape:
            scratch.params['beta0'] = bn_model.bias.data
            with open('./weight_parameter/bn_param/bias/{}'.format(0), mode='w') as f:
                f.write(str(bn_model.bias.data))
        elif bn_model.bias.data.shape == scratch.params['beta1'].shape:
            scratch.params['beta1'] = bn_model.bias.data
            with open('./weight_parameter/bn_param/bias/{}'.format(1), mode='w') as f:
                f.write(str(bn_model.bias.data))
        elif bn_model.bias.data.shape == scratch.params['beta2'].shape:
            scratch.params['beta2'] = bn_model.bias.data
            with open('./weight_parameter/bn_param/bias/{}'.format(2), mode='w') as f:
                f.write(str(bn_model.bias.data))
        elif bn_model.bias.data.shape == scratch.params['beta3'].shape:
            scratch.params['beta3'] = bn_model.bias.data
            with open('./weight_parameter/bn_param/bias/{}'.format(3), mode='w') as f:
                f.write(str(bn_model.bias.data))
        elif bn_model.bias.data.shape == scratch.params['beta4'].shape:
            scratch.params['beta4'] = bn_model.bias.data
            with open('./weight_parameter/bn_param/bias/{}'.format(4), mode='w') as f:
                f.write(str(bn_model.bias.data))
        elif bn_model.bias.data.shape == scratch.params['beta5'].shape:
            scratch.params['beta5'] = bn_model.bias.data
            with open('./weight_parameter/bn_param/bias/{}'.format(5), mode='w') as f:
                f.write(str(bn_model.bias.data))
        elif (bn_model.bias.data.shape == scratch.params['beta6'].shape) and self.b == "b":
            scratch.params['beta6'] = bn_model.bias.data
            with open('./weight_parameter/bn_param/bias/{}'.format(6), mode='w') as f:
                f.write(str(bn_model.bias.data))
            self.b = 'bb'
        elif (scratch.params['beta7'].shape == bn_model.bias.data.shape) and self.b == "bb":
            scratch.params['beta7'] = bn_model.bias.data
            with open('./weight_parameter/bn_param/bias/{}'.format(7), mode='w') as f:
                f.write(str(bn_model.bias.data))

        self.start = self.start + num_b

        

        bn_model.weight.data.copy_(
            torch.reshape(torch.from_numpy(self.buf[self.start:self.start + num_b]), bn_model.bias.size()))


        if bn_model.weight.data.shape == scratch.params['gamma0'].shape:
            scratch.params['gamma0'] = bn_model.weight.data
            with open('./weight_parameter/bn_param/gamma/{}'.format(0), mode='w') as f:
                f.write(str(bn_model.weight.data))
        elif bn_model.weight.data.shape == scratch.params['gamma1'].shape:
            scratch.params['gamma1'] = bn_model.weight.data
            with open('./weight_parameter/bn_param/gamma/{}'.format(1), mode='w') as f:
                f.write(str(bn_model.weight.data))
        elif bn_model.weight.data.shape == scratch.params['gamma2'].shape:
            scratch.params['gamma2'] = bn_model.weight.data
            with open('./weight_parameter/bn_param/gamma/{}'.format(2), mode='w') as f:
                f.write(str(bn_model.weight.data))
        elif bn_model.weight.data.shape == scratch.params['gamma3'].shape:
            scratch.params['gamma3'] = bn_model.weight.data
            with open('./weight_parameter/bn_param/gamma/{}'.format(3), mode='w') as f:
                f.write(str(bn_model.weight.data))
        elif bn_model.weight.data.shape == scratch.params['gamma4'].shape:
            scratch.params['gamma4'] = bn_model.weight.data
            with open('./weight_parameter/bn_param/gamma/{}'.format(4), mode='w') as f:
                f.write(str(bn_model.weight.data))
        elif bn_model.weight.data.shape == scratch.params['gamma5'].shape:
            scratch.params['gamma5'] = bn_model.weight.data
            with open('./weight_parameter/bn_param/gamma/{}'.format(5), mode='w') as f:
                f.write(str(bn_model.weight.data))
        elif (bn_model.weight.shape == scratch.params['gamma6'].shape) and self.g == "g":
            scratch.params['gamma6'] = bn_model.weight.data
            with open('./weight_parameter/bn_param/gamma/{}'.format(6), mode='w') as f:
                f.write(str(bn_model.weight.data))
            self.g = 'gg'
        elif (scratch.params['gamma7'].shape == bn_model.weight.data.shape) and self.g == "gg":
            scratch.params['gamma7'] = bn_model.weight.data
            with open('./weight_parameter/bn_param/gamma/{}'.format(7), mode='w') as f:
                f.write(str(bn_model.weight.data))

        self.start = self.start + num_b

        bn_model.running_mean.copy_(
            torch.reshape(torch.from_numpy(self.buf[self.start:self.start + num_b]), bn_model.bias.size()))

        if bn_model.running_mean.data.shape == scratch.bn_params[0]['running_mean'].shape:
            scratch.bn_params[0]['running_mean'] = bn_model.running_mean.data
            with open('./weight_parameter/bn_param/running_mean/{}'.format(0), mode='w') as f:
                f.write(str(bn_model.running_mean.data))
        elif bn_model.running_mean.data.shape == scratch.bn_params[1]['running_mean'].shape:
            scratch.bn_params[1]['running_mean'] = bn_model.running_mean.data
            with open('./weight_parameter/bn_param/running_mean/{}'.format(1), mode='w') as f:
                f.write(str(bn_model.running_mean.data))
        elif bn_model.running_mean.data.shape == scratch.bn_params[2]['running_mean'].shape:
            scratch.bn_params[2]['running_mean'] = bn_model.running_mean.data
            with open('./weight_parameter/bn_param/running_mean/{}'.format(2), mode='w') as f:
                f.write(str(bn_model.running_mean.data))
        elif bn_model.running_mean.data.shape == scratch.bn_params[3]['running_mean'].shape:
            scratch.bn_params[3]['running_mean'] = bn_model.running_mean.data
            with open('./weight_parameter/bn_param/running_mean/{}'.format(3), mode='w') as f:
                f.write(str(bn_model.running_mean.data))
        elif bn_model.running_mean.data.shape == scratch.bn_params[4]['running_mean'].shape:
            scratch.bn_params[4]['running_mean'] = bn_model.running_mean.data
            with open('./weight_parameter/bn_param/running_mean/{}'.format(4), mode='w') as f:
                f.write(str(bn_model.running_mean.data))
        elif bn_model.running_mean.data.shape == scratch.bn_params[5]['running_mean'].shape:
            scratch.bn_params[5]['running_mean'] = bn_model.running_mean.data
            with open('./weight_parameter/bn_param/running_mean/{}'.format(5), mode='w') as f:
                f.write(str(bn_model.running_mean.data))
        elif bn_model.running_mean.data.shape == scratch.bn_params[6]['running_mean'].shape and self.rm == "rm":
            scratch.bn_params[6]['running_mean'] = bn_model.running_mean.data
            with open('./weight_parameter/bn_param/running_mean/{}'.format(6), mode='w') as f:
                f.write(str(bn_model.running_mean.data))
            self.rm = "rmrm"
        elif bn_model.running_mean.data.shape == scratch.bn_params[7]['running_mean'].shape and self.rm == "rmrm":
            scratch.bn_params[7]['running_mean'] = bn_model.running_mean.data
            with open('./weight_parameter/bn_param/running_mean/{}'.format(7), mode='w') as f:
                f.write(str(bn_model.running_mean.data))

        self.start = self.start + num_b

        bn_model.running_var.copy_(
            torch.reshape(torch.from_numpy(self.buf[self.start:self.start + num_b]), bn_model.bias.size()))

        if bn_model.running_var.data.shape == scratch.bn_params[0]['running_var'].shape:
            scratch.bn_params[0]['running_var'] = bn_model.running_var.data
            with open('./weight_parameter/bn_param/running_var/{}'.format(0), mode='w') as f:
                f.write(str(bn_model.running_var.data))
        elif bn_model.running_var.data.shape == scratch.bn_params[1]['running_var'].shape:
            scratch.bn_params[1]['running_var'] = bn_model.running_var.data
            with open('./weight_parameter/bn_param/running_var/{}'.format(1), mode='w') as f:
                f.write(str(bn_model.running_var.data))
        elif bn_model.running_var.data.shape == scratch.bn_params[2]['running_var'].shape:
            scratch.bn_params[2]['running_var'] = bn_model.running_var.data
            with open('./weight_parameter/bn_param/running_var/{}'.format(2), mode='w') as f:
                f.write(str(bn_model.running_var.data))
        elif bn_model.running_var.data.shape == scratch.bn_params[3]['running_var'].shape:
            scratch.bn_params[3]['running_var'] = bn_model.running_var.data
            with open('./weight_parameter/bn_param/running_var/{}'.format(3), mode='w') as f:
                f.write(str(bn_model.running_var.data))
        elif bn_model.running_var.data.shape == scratch.bn_params[4]['running_var'].shape:
            scratch.bn_params[4]['running_var'] = bn_model.running_var.data
            with open('./weight_parameter/bn_param/running_var/{}'.format(4), mode='w') as f:
                f.write(str(bn_model.running_var.data))
        elif bn_model.running_var.data.shape == scratch.bn_params[5]['running_var'].shape:
            scratch.bn_params[5]['running_var'] = bn_model.running_var.data
            with open('./weight_parameter/bn_param/running_var/{}'.format(5), mode='w') as f:
                f.write(str(bn_model.running_var.data))
        elif bn_model.running_var.data.shape == scratch.bn_params[6]['running_var'].shape and self.rv == "rv":
            scratch.bn_params[6]['running_var'] = bn_model.running_var.data
            with open('./weight_parameter/bn_param/running_var/{}'.format(6), mode='w') as f:
                f.write(str(bn_model.running_var.data))
            self.rv = "rvrv"
        elif bn_model.running_var.data.shape == scratch.bn_params[7]['running_var'].shape and self.rv == "rvrv":
            scratch.bn_params[7]['running_var'] = bn_model.running_var.data
            with open('./weight_parameter/bn_param/running_var/{}'.format(7), mode='w') as f:
                f.write(str(bn_model.running_var.data))
            
        self.start = self.start + num_b

        conv_model.weight.data.copy_(
            torch.reshape(torch.from_numpy(self.buf[self.start:self.start + num_w]), conv_model.weight.size()))
        
        
        if conv_model.weight.data.shape == (16, 3, 3, 3):
            scratch.params['W0'] = conv_model.weight.data
            with open('./weight_parameter/conv_param/w/{}'.format(0), mode='w') as f:
                f.write(str(conv_model.weight.data))
        elif conv_model.weight.data.shape == (32, 16, 3, 3):
            scratch.params['W1'] = conv_model.weight.data
            with open('./weight_parameter/conv_param/w/{}'.format(1), mode='w') as f:
                f.write(str(conv_model.weight.data))
        elif conv_model.weight.data.shape == (64, 32, 3, 3):
            scratch.params['W2'] = conv_model.weight.data
            with open('./weight_parameter/conv_param/w/{}'.format(2), mode='w') as f:
                f.write(str(conv_model.weight.data))
        elif conv_model.weight.data.shape == (128, 64, 3, 3):
            scratch.params['W3'] = conv_model.weight.data
            with open('./weight_parameter/conv_param/w/{}'.format(3), mode='w') as f:
                f.write(str(conv_model.weight.data))
        elif conv_model.weight.data.shape == (256, 128, 3, 3):
            scratch.params['W4'] = conv_model.weight.data
            with open('./weight_parameter/conv_param/w/{}'.format(4), mode='w') as f:
                f.write(str(conv_model.weight.data))
        elif conv_model.weight.data.shape == (512, 256, 3, 3):
            scratch.params['W5'] = conv_model.weight.data
            with open('./weight_parameter/conv_param/w/{}'.format(5), mode='w') as f:
                f.write(str(conv_model.weight.data))
        elif conv_model.weight.data.shape == (1024, 512, 3, 3):            
            scratch.params['W6'] = conv_model.weight.data
            with open('./weight_parameter/conv_param/w/{}'.format(6), mode='w') as f:
                f.write(str(conv_model.weight.data))
        elif conv_model.weight.data.shape == (1024, 1024, 3, 3):
            scratch.params['W7'] = conv_model.weight.data
            with open('./weight_parameter/conv_param/w/{}'.format(7), mode='w') as f:
                f.write(str(conv_model.weight.data))
        self.start = self.start + num_w

    def load_conv(self, conv_model):
        num_w = conv_model.weight.numel()
        num_b = conv_model.bias.numel()
        conv_model.bias.data.copy_(
            torch.reshape(torch.from_numpy(self.buf[self.start:self.start + num_b]), conv_model.bias.size()))
        scratch.params['b8'] = conv_model.bias.data
        with open('./weight_parameter/bias/{}'.format(7), mode='w') as f:
            f.write(str(conv_model.bias.data))
        self.start = self.start + num_b
        conv_model.weight.data.copy_(
            torch.reshape(torch.from_numpy(self.buf[self.start:self.start + num_w]), conv_model.weight.size()))
        scratch.params['W8'] = conv_model.weight.data
        with open('./weight_parameter/conv_param/w/{}'.format(8), mode='w') as f:
            f.write(str(conv_model.weight.data))
        self.start = self.start + num_w

    def dfs(self, m):
        children = list(m.children())
        for i, c in enumerate(children):
            if isinstance(c, torch.nn.Sequential):
                self.dfs(c)
            elif isinstance(c, torch.nn.Conv2d):
                if c.bias is not None:
                    self.load_conv(c)
                else:
                    self.load_conv_bn(c, children[i + 1])

    def load(self, model, weights_file):
        self.start = 0
        fp = open(weights_file, 'rb')
        header = np.fromfile(fp, count=4, dtype=np.int32)
        self.buf = np.fromfile(fp, dtype=np.float32)
        fp.close()
        size = self.buf.size
        self.dfs(model)
        # make sure the loaded weight is right
    
        assert size == self.start

In [ ]:
from weight_loader import WeightLoader

In [ ]:
scratch = DeepConvNet(input_dims=(3, 416, 416),
                        num_filters=[16, 32, 64, 128, 256, 512, 1024, 1024],
                        max_pools=[0, 1, 2, 3, 4],
                        weight_scale='kaiming',
                        batchnorm=True,
                        dtype=torch.float32, device='cpu')

In [ ]:
model = Yolov2()

In [ ]:
weightloader = WeightLoader()
weightloader.load(model, './yolov2-tiny-voc.weights')

In [ ]:
x = torch.rand(1, 3, 416, 416)

In [ ]:
output_scratch = scratch.loss(x)

In [ ]:
output_torch = model(x)

In [ ]:
print(eecs598.grad.rel_error(output_scratch[0], output_torch[0]))

In [ ]:
torch.manual_seed(42)

# Check Conv

In [15]:
PythonConv = ConvB
TorchConv = FastConvWB

In [11]:
device = 'cuda' if torch.cuda.is_available() else "cpu"

In [12]:
torch.manual_seed(42)
x = torch.randn(4, 3, 5, 5, dtype=torch.float64, device='cuda')
w = torch.randn(2, 3, 3, 3, dtype=torch.float64, device='cuda')
b = torch.randn(2, dtype=torch.float64, device='cuda')
dout = torch.randn(4, 2, 5, 5, dtype=torch.float64, device='cuda')
conv_param = {'stride': 1, 'pad': 1}

dx_num = eecs598.grad.compute_numeric_gradient(lambda x: ConvB.forward(x, w, b, conv_param)[0], x, dout)
dw_num = eecs598.grad.compute_numeric_gradient(lambda w: ConvB.forward(x, w, b, conv_param)[0], w, dout)
db_num = eecs598.grad.compute_numeric_gradient(lambda b: ConvB.forward(x, w, b, conv_param)[0], b, dout)

out, cache = ConvB.forward(x, w, b, conv_param)
dx, dw, db = ConvB.backward(dout, cache)

out2, cache2 = FastConvWB.forward(x, w, b, conv_param)
dx2, dw2, db2 = FastConvWB.backward(dout, cache2)

print('Testing Conv.backward function')
print('dx error: ', eecs598.grad.rel_error(dx, dx2))
print('dw error: ', eecs598.grad.rel_error(dw, dw2))
print('db error: ', eecs598.grad.rel_error(db, db2))

torch.Size([4, 3, 5, 5])
Testing Conv.backward function
dx error:  7.937759210108112e-17
dw error:  8.086311976557575e-17
db error:  1.643682260085236e-16


In [67]:
torch.manual_seed(42)
input 			= torch.randn(4, 3, 5, 5, dtype=torch.float64, device='cuda')
weights			= torch.randn(2, 3, 3, 3, dtype=torch.float64, device='cuda')
bias 			= torch.randn(2, dtype=torch.float64, device='cuda')
loss_grads		= torch.randn(4, 2, 5, 5, dtype=torch.float64, device='cuda')
conv_param 		= {'stride': 1, 'pad': 1}

# Forward Propagation
out, cache 		= PythonConv.forward(input, weights, bias, conv_param)
out2, cache2 	= TorchConv.forward( input, weights, bias, conv_param)

# Loss Calculation
# dummy loss gradient - generated randomly

# Backward Propagation
dx, dw, db 		= PythonConv.backward(dout, cache)
dx2, dw2, db2 	= TorchConv.backward(dout, cache2)

# Check the output of Forward
print('\nTesting Conv.forward function')
print('\tOut error: ', eecs598.grad.rel_error(out, out2))

# Check the output of Backward
print('\nTesting Conv.backward function')
print('\tdx error: ', eecs598.grad.rel_error(dx, dx2))
print('\tdw error: ', eecs598.grad.rel_error(dw, dw2))
print('\tdb error: ', eecs598.grad.rel_error(db, db2))

torch.Size([4, 3, 5, 5])

Testing Conv.forward function
	Out error:  1.4868675061790858e-16

Testing Conv.backward function
	dx error:  7.937759210108112e-17
	dw error:  8.086311976557575e-17
	db error:  1.643682260085236e-16


In [68]:
print(input.shape)
print(weights.shape)
print(bias.shape)

torch.Size([4, 3, 5, 5])
torch.Size([2, 3, 3, 3])
torch.Size([2])


In [102]:
# del input, loss_grads, bias, weights
loss_grads 	= pickle.load(open('../Temp_Files/Python/Backward_loss_gradients.pickle', 'rb'))
input 		= pickle.load(open('../Temp_Files/Python/Forward_Out.pickle', 'rb'))[7]
params  	= pickle.load(open('../paramters.pickle', 'rb'))
weights		= params["W8"]
bias 		= params["b8"]

torch.manual_seed(42)
input			= input.to(device)
weights			= weights.to(device)
bias 			= bias.to(device)
loss_grads 		= loss_grads.to(device)
conv_param 		= {'stride': 1, 'pad': 1}

In [95]:
print(f"input	: {input.shape}")
print(f"weights	: {weights.shape}")
print(f"bias	: {bias.shape}")

print(f"input	: {input.device}")
print(f"weights	: {weights.device}")
print(f"bias	: {bias.device}")

input	: torch.Size([1, 1024, 13, 13])
weights	: torch.Size([125, 1024, 1, 1])
bias	: torch.Size([125])
input	: cuda:0
weights	: cuda:0
bias	: cuda:0


In [97]:
# Forward Propagation
out, cache 		= PythonConv.forward(input, weights, bias, conv_param)
out2, cache2 	= TorchConv.forward( input, weights, bias, conv_param)
print(f"{out[0,:10,0,0]}\n\n{out2[0,:10,0,0]}")

In [105]:
# Loss Calculation
# Loss Gradient taken from code
print(loss_grads[0,:10,0,0])

tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.4574e-08, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00], device='cuda:0')


In [107]:
# Backward Propagation
dx, dw, db 		= PythonConv.backward(loss_grads, cache)
dx2, dw2, db2 	= TorchConv.backward (loss_grads, cache2)

torch.Size([1, 1024, 13, 13])


In [108]:
# Check the output of Forward
print('\nTesting Conv.forward function')
print('\tOut error: ', eecs598.grad.rel_error(out, out2))

# Check the output of Backward
print('\nTesting Conv.backward function')
print('\tdx error: ', eecs598.grad.rel_error(dx, dx2))
print('\tdw error: ', eecs598.grad.rel_error(dw, dw2))
print('\tdb error: ', eecs598.grad.rel_error(db, db2))


Testing Conv.forward function
	Out error:  5.382756366892442e-07

Testing Conv.backward function
	dx error:  1.0
	dw error:  1.0
	db error:  1.0


In [ ]:
PythonLay0  = pickle.load(open('../Temp_Files/Python/Backward_loss_gradients.pickle', 'rb'))
PytorchLay0 = pickle.load(open('../Temp_Files/Python/Backward_loss_gradients.pickle', 'rb'))
print('\tdb error: ', eecs598.grad.rel_error(db, db2))

In [ ]:
print(dx[0, 0, 0, :], dx2[0, 0, 0, :])

In [ ]:
torch.manual_seed(42)
x = torch.rand(1, 1, 12, 12)
w = torch.randn(3, 1, 3, 3)
nn.init.ones_(w)
conv_param = {'stride': 2, 'pad': 1}
out, cache = Conv.forward(x, w, conv_param)
# model = nn.Conv2d(1, 3, kernel_size=3, stride=2, padding=1, bias=False)
# if isinstance(model, nn.Conv2d):
#     nn.init.ones_(model.weight)
out2, cache2 = FastConv.forward(x, w, conv_param)
eecs598.grad.rel_error(out, out2)

In [ ]:
x, w, conv_param = cache
x2, w2, conv_param, tx, out2, layer = cache2

In [ ]:
eecs598.grad.rel_error(x, x2)

# Check Conv_Relu

In [ ]:
x = torch.rand(1, 1, 12, 12)
w = torch.randn(3, 1, 3, 3)
nn.init.ones_(w)
conv_param = {'stride': 2, 'pad': 1}

out, _ = Conv_ReLU.forward(x, w, conv_param)
y = nn.Sequential(
    nn.Conv2d(1, 3, kernel_size=3, stride=2, padding=1, bias=False),
    nn.LeakyReLU(0.1)
    )
for k in y.modules():
    if isinstance(k, nn.Conv2d):
        nn.init.ones_(k.weight)
out2 = y(x)
eecs598.grad.rel_error(out, out2)

In [ ]:
x = torch.rand(1, 1, 12, 12)
w = torch.randn(3, 1, 3, 3)
nn.init.ones_(w)
conv_param = {'stride': 2, 'pad': 1}
pool_param = {'pool_height': 2, 'pool_width': 2, 'stride': 2}

out, _ = Conv_ReLU_Pool.forward(x, w, conv_param, pool_param)
y = nn.Sequential(
    nn.Conv2d(1, 3, kernel_size=3, stride=2, padding=1, bias=False),
    nn.LeakyReLU(0.1),
    nn.MaxPool2d(2, 2)
    )
for k in y.modules():
    if isinstance(k, nn.Conv2d):
        nn.init.ones_(k.weight)
out2 = y(x)
eecs598.grad.rel_error(out, out2)

# Check Conv_BatchNorm_ReLU

In [ ]:
x = torch.rand(1, 1, 12, 12)
w = torch.randn(3, 1, 3, 3)
nn.init.ones_(w)
conv_param = {'stride': 2, 'pad': 1}
pool_param = {'pool_height': 2, 'pool_width': 2, 'stride': 2}
gamma = torch.ones(3)
beta = torch.ones(3)
bn_param = {'mode': 'train',
            'running_mean': torch.ones(3),
            'running_var': torch.ones(3)}

out, _ = Conv_BatchNorm_ReLU.forward(x, w, gamma, beta, conv_param, bn_param)
y = nn.Sequential(
    nn.Conv2d(1, 3, kernel_size=3, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(3),
    nn.LeakyReLU(0.1),
    )
for k in y.modules():
    if isinstance(k, nn.Conv2d):
        nn.init.ones_(k.weight)
    if isinstance(k, nn.BatchNorm2d):
        nn.init.ones_(k.weight)
        nn.init.ones_(k.bias)
out2 = y(x)
eecs598.grad.rel_error(out, out2)

# Check Conv_BatchNorm_ReLU_Pool

In [ ]:
x = torch.rand(1, 1, 12, 12)
w = torch.randn(3, 1, 3, 3)
nn.init.ones_(w)
conv_param = {'stride': 2, 'pad': 1}
pool_param = {'pool_height': 2, 'pool_width': 2, 'stride': 2}
gamma = torch.ones(3)
beta = torch.ones(3)
bn_param = {'mode': 'train',
            'running_mean': torch.ones(3),
            'running_var': torch.ones(3)}

out, _ = Conv_BatchNorm_ReLU_Pool.forward(x, w, gamma, beta, conv_param, bn_param, pool_param)
y = nn.Sequential(
    nn.Conv2d(1, 3, kernel_size=3, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(3),
    nn.LeakyReLU(0.1),
    nn.MaxPool2d(2, 2)
    )
for k in y.modules():
    if isinstance(k, nn.Conv2d):
        nn.init.ones_(k.weight)
    if isinstance(k, nn.BatchNorm2d):
        nn.init.ones_(k.weight)
        nn.init.ones_(k.bias)
out2 = y(x)
eecs598.grad.rel_error(out, out2)